## Data Preprocessing Clean  tweets


- **Nettoyage du texte**

    - Remove URLs, mentions (@username), hashtags (or keep them if relevant)
    - Handle emojis (remove or convert to text)
    - Lowercase text
    - Remove extra whitespace
    - ponctuation

- **Embedding**

- **Save metadata**

---


In [45]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import re

In [46]:
df = pd.read_csv("../data/raw/data.csv")

df.head()


,airline_sentiment,negativereason,airline,text,tweet_created
0,neutral,NaN,Virgin America,@VirginAmerica What @dhepburn said.,2/24/2015 11:35
1,positive,NaN,Virgin America,@VirginAmerica plus you've added commercials t...,2/24/2015 11:15
2,neutral,NaN,Virgin America,@VirginAmerica I didn't today... Must mean I n...,2/24/2015 11:15
3,negative,Bad Flight,Virgin America,@VirginAmerica it's really aggressive to blast...,2/24/2015 11:15
4,negative,Can't Tell,Virgin America,@VirginAmerica and it's a really big bad thing...,2/24/2015 11:14


In [47]:

def preprocess_text(text):
    """
    Preprocess text for transformer models
    - Remove HTML tags
    - Remove URLs
    - Remove mentions (@username)
    - Remove hashtags (#hashtag -> hashtag or removed)
    - Normalize whitespace
    """

    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Hashtags often carry important context , just remove #
    # Remove hashtags (keep the word, remove #)
    text = re.sub(r'#(\w+)', r'\1', text)
    # remove hashtag with word
    # text = re.sub(r'#\w+', '', text)


    # Remove special characters but keep basic punctuation and letters
    text = re.sub(r'[^\w\s.,!?;:\-\'"()]', ' ', text)


    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text



df["clean_text"] = df['text'].apply(preprocess_text)


# null values after cleaning
initial_count = len(df)
df = df[df['clean_text'].str.strip() != '']
print(f"Removed {initial_count - len(df)} empty texts after cleaning")


# drop duplicates text after preprocessing
df = df.drop_duplicates(subset=["clean_text"])

# Reset index
df = df.reset_index(drop=True)

df

Removed 0 empty texts after cleaning


,airline_sentiment,negativereason,airline,text,tweet_created,clean_text
0,neutral,NaN,Virgin America,@VirginAmerica What @dhepburn said.,2/24/2015 11:35,What said.
1,positive,NaN,Virgin America,@VirginAmerica plus you've added commercials t...,2/24/2015 11:15,plus you've added commercials to the experienc...
2,neutral,NaN,Virgin America,@VirginAmerica I didn't today... Must mean I n...,2/24/2015 11:15,I didn't today... Must mean I need to take ano...
3,negative,Bad Flight,Virgin America,@VirginAmerica it's really aggressive to blast...,2/24/2015 11:15,"it's really aggressive to blast obnoxious ""ent..."
4,negative,Can't Tell,Virgin America,@VirginAmerica and it's a really big bad thing...,2/24/2015 11:14,and it's a really big bad thing about it
...,...,...,...,...,...,...
14268,positive,NaN,American,@AmericanAir thank you we got on a different f...,2/22/2015 12:01,thank you we got on a different flight to Chic...
14269,negative,Customer Service Issue,American,@AmericanAir leaving over 20 minutes Late Flig...,2/22/2015 11:59,leaving over 20 minutes Late Flight. No warnin...
14270,neutral,NaN,American,@AmericanAir Please bring American Airlines to...,2/22/2015 11:59,Please bring American Airlines to BlackBerry10
14271,negative,Customer Service Issue,American,"@AmericanAir you have my money, you change my ...",2/22/2015 11:59,"you have my money, you change my flight, and d..."


In [48]:
print(f"\nDataset shapes after preprocessing:")
print(f"Train: {df.shape}")

print(f"\nDuplicates after preprocessing:")
print(f"Train: {df['clean_text'].duplicated().sum()}")

print(f"\nNull alues after preprocessing:")
df.isnull().sum()


Dataset shapes after preprocessing:
Train: (14273, 6)

Duplicates after preprocessing:
Train: 0

Null alues after preprocessing:


airline_sentiment       0
negativereason       5204
airline                 0
text                    0
tweet_created           0
clean_text              0
dtype: int64

In [50]:

import os
os.makedirs("../data/processed")

df.to_csv("../data/processed/data.csv")

preprocess_text("df saved")

'df saved'

In [ ]:
print(df['airline_sentiment'].unique())

['neutral' 'positive' 'negative']


In [ ]:
# Load the model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Prepare your data
X = df['clean_text'].values
y = df['airline_sentiment'].values

# embeddings
X_embeddings = model.encode(X, show_progress_bar=True, batch_size=32)



Batches: 100%|██████████| 447/447 [01:20<00:00,  5.56it/s]


Embedding shape: (14273, 384)


In [ ]:
X

In [ ]:
print(f"Embedding shape: {X_embeddings.shape}") 
print(f"Each text is represented as a {X_embeddings.shape[1]}-dimensional vector")

Embedding shape: (14273, 384)
Each text is represented as a 384-dimensional vector


In [ ]:
import os
os.makedirs("../data/embedding",exist_ok=True)

# Save embeddings
np.save('../data/embedding/embeddings.npy', X_embeddings)
print("Embeddings saved!")

Embeddings saved!


In [ ]:
print(f"\nEmbeddings shape: {X_embeddings.shape}")
print(f"Each text is represented as a {X_embeddings.shape[1]}-dimensional vector")


Embeddings shape: (14273, 384)
Each text is represented as a 384-dimensional vector


### Meta Data

In [ ]:
os.makedirs("../data/metadata",exist_ok=True)

# add more info
train_metadata = pd.DataFrame({
    'id': df.index,
    'airline':df['airline'],
    'text': df['text'],
    'clean_text': df['clean_text'],
    'negativereason':df['negativereason'],
    'tweet_created':df['tweet_created'],
    'label': df['airline_sentiment'],
})

train_metadata.to_csv('../data/metadata/metadata.csv', index=False)
